# Porespy analysis of the 'framed' scan

In [ ]:
# General notebook settings
import seaborn
# Set seaborn theme
seaborn.set_theme(context='notebook', style='ticks')

In [ ]:
plt.rcParams['figure.figsize']

In [ ]:
# Set figure defaults
plt.rc('image', cmap='gray', interpolation='nearest')  # Display all images in b&w and with 'nearest' interpolation
# scalefactor = 2
# plt.rcParams['figure.figsize'] = (16 // scalefactor, 9 // scalefactor)  # Size up figures a bit
# plt.rcParams['figure.dpi'] = 300

In [ ]:
# Set figure defaults
plt.rc('image', cmap='gray', interpolation='nearest')  # Display all images in b&w and with 'nearest' interpolation
scalefactor = 2
plt.rcParams['figure.figsize'] = (16 // scalefactor, 9 // scalefactor)  # Size up figures a bit
plt.rcParams['figure.dpi'] = 300

In [ ]:
# Necessary imports
import platform
import os
import pandas
import glob
import pathlib
from tqdm.auto import tqdm, trange
import dask_image.imread

In [ ]:
# Load our own log file parsing code
# This is loaded as a submodule to alleviate excessive copy-pasting between *all* projects we do
# See https://github.com/habi/BrukerSkyScanLogfileRuminator for details on its inner workings
from BrukerSkyScanLogfileRuminator.parsing_functions import *

In [ ]:
if 'Win' in platform.system():
    Root = 'F:/'
else:
    Root = '/media/habi/Fast_SSD'
Path = os.path.join(Root, 'Schmid BFH Methylcellulose')
print('Our base path is %s' % Path)

In [ ]:
# Make us a dataframe for saving all that we need
Data = pandas.DataFrame()

In [ ]:
# Get *all* log files present on disk
# Using os.walk is way faster than using recursive glob.glob
# Not sorting the found logfiles is also making it quicker
Data['LogFile'] = [os.path.join(root, name)
                   for root, dirs, files in os.walk(Path)
                   for name in files
                   if name.endswith((".log"))]

In [ ]:
# Get all folders
Data['Folder'] = [os.path.dirname(f) for f in Data['LogFile']]
Data['FolderShort'] = [folder[len(Root) + 1:] for folder in Data['Folder']]

In [ ]:
Data.sample(n=5)

In [ ]:
# Get rid of all the logfiles from all the folders that might be on disk but that we don't want to load the data from
for c, row in Data.iterrows():
    if 'proj' in os.path.split(row.Folder)[-1]:  # drop all projections folders
        Data.drop([c], inplace=True)
    if os.path.split(row.Folder)[-1] == 'PR':  # drop all phase retrieval folders for the moment
        Data.drop([c], inplace=True)        
    elif 'rectmp.log' in row.LogFile:  # drop temporary log files of samples currently being reconstructed
        Data.drop([c], inplace=True)
# Reset dataframe to something that we would get if we only would have loaded the 'rec' files
Data = Data.reset_index(drop=True)

In [ ]:
# Generate us some meaningful colums in the dataframe
Data['Sample'] = [('-').join([pathlib.Path(log).parts[-4], pathlib.Path(log).parts[-3]]) for log in Data['LogFile']]
Data['Scan'] = [os.path.basename(os.path.dirname(log)) for log in Data['LogFile']]

In [ ]:
# Load the file names of all the reconstructions of all the scans
Data['Filenames Reconstructions'] = [sorted(glob.glob(os.path.join(f, '*rec0*.png'))) for f in Data['Folder']]
# How many reconstructions do we have?
Data['Number of reconstructions'] = [len(r) for r in Data['Filenames Reconstructions']]

In [ ]:
# Drop samples which have either not been reconstructed yet or of which we deleted the reconstructions with
# `find . -name "*rec*.png" -type f -mtime +333 -delete`
# Based on https://stackoverflow.com/a/13851602
# for c,row in Data.iterrows():
#     if not row['Number of reconstructions']:
#         print('%s contains no PNG files, we might be currently reconstructing it' % row.Folder)
Data = Data[Data['Number of reconstructions'] > 0]
# Reset the dataframe count/index for easier indexing afterwards
Data.reset_index(drop=True, inplace=True)
print('We have %s folders with reconstructions' % (len(Data)))

In [ ]:
# Get parameters to doublecheck from logfiles
Data['Voxelsize'] = [pixelsize(log) for log in Data['LogFile']]
Data['Camera'] = [camera(log) for log in Data['LogFile']]
Data['Filter'] = [whichfilter(log) for log in Data['LogFile']]
Data['Exposuretime'] = [exposuretime(log) for log in Data['LogFile']]
Data['Scanner'] = [scanner(log) for log in Data['LogFile']]
Data['Averaging'] = [averaging(log) for log in Data['LogFile']]
Data['ProjectionSize'] = [projection_size(log) for log in Data['LogFile']]
Data['RotationStep'] = [rotationstep(log) for log in Data['LogFile']]
Data['Grayvalue'] = [reconstruction_grayvalue(log) for log in Data['LogFile']]
Data['RingartefactCorrection'] = [ringremoval(log) for log in Data['LogFile']]
Data['BeamHardeningCorrection'] = [beamhardening(log) for log in Data['LogFile']]
Data['DefectPixelMasking'] = [defectpixelmasking(log) for log in Data['LogFile']]
Data['Scan date'] = [scandate(log) for log in Data['LogFile']]

In [ ]:
# Get rid of all the scans except 'Framed'
for c, row in Data.iterrows():
    if 'Blobs' in row.LogFile or 'Chunks' in row.LogFile or 'Kreidegrund' in row.LogFile:
        Data.drop([c], inplace=True)
# Reset dataframe to something that we would get if we only would have loaded the 'rec' files
Data = Data.reset_index(drop=True)

In [ ]:
Data.Sample.unique()

In [ ]:
# # Load all reconstructions DASK arrays
# Reconstructions = [dask_image.imread.imread(os.path.join(folder,'*rec*.png')) for folder in Data['Folder']]
# Load all reconstructions into ephemereal DASK arrays, with a nice progress bar...
Reconstructions = [None] * len(Data)
for c, row in tqdm(Data.iterrows(),
                   desc='Loading reconstructions',
                   total=len(Data)):
    Reconstructions[c] = dask_image.imread.imread(os.path.join(row['Folder'], '*rec*.png'))[:,:,:,0]

In [ ]:
Reconstructions[0]

In [ ]:
# Load test image
inputimage = Reconstructions[0][Reconstructions[0].shape[0]//2].compute()

In [ ]:
from matplotlib_scalebar.scalebar import ScaleBar
# Setup scale bar defaults
plt.rcParams['scalebar.location'] = 'lower right'
plt.rcParams['scalebar.frameon'] = False
plt.rcParams['scalebar.color'] = 'white'

In [ ]:
import skimage

In [ ]:
# Show test image
plt.imshow(skimage.exposure.equalize_adapthist(inputimage))
plt.title('%s, %s x %s px' % (os.path.basename(Data['Filenames Reconstructions'][0][len(Data['Filenames Reconstructions'][0])//2]),
                              inputimage.shape[0],
                              inputimage.shape[1]))
plt.gca().add_artist(ScaleBar(Data['Voxelsize'][0],'um'))
plt.axis('off')
plt.show()

In [ ]:
# Use only central part of the image
crop = 200
plt.subplot(121)
plt.imshow(inputimage)
plt.axhline(crop)
plt.axhline(inputimage.shape[0]-crop)
plt.axvline(crop)
plt.axvline(inputimage.shape[1]-crop)
plt.gca().add_artist(ScaleBar(voxelsize,'um'))
plt.axis('off')
plt.subplot(122)
croppedimage = inputimage[crop:-crop,crop:-crop]
plt.imshow(skimage.exposure.equalize_adapthist(croppedimage))
plt.title('Middle slice crop, %s x %s px' % (croppedimage.shape[0], croppedimage.shape[1]))
plt.gca().add_artist(ScaleBar(voxelsize,'um'))
plt.axis('off')
plt.show()

In [ ]:
# Calculate thresholds to separate into foam and background
threshold_iso = skimage.filters.threshold_isodata(croppedimage)
threshold_otsu = skimage.filters.threshold_otsu(croppedimage)

In [ ]:
# Display gray value histogram of image
histogram = plt.hist(croppedimage.ravel(),
                     bins='doane', # nice bin size selection
                     histtype='bar',
                     log=True,
                     label='Histogram',
                     color=seaborn.color_palette()[0])
plt.axvline(threshold_iso, label='Isodata-Threshold@%s' % threshold_iso, c=seaborn.color_palette()[1])
plt.axvline(threshold_otsu, label='Otsu-Threshold@%s' % threshold_otsu, c=seaborn.color_palette()[2])
plt.legend()
plt.title('Logarithmic grayvalue histogram with %s bins' % len(histogram[1]))
seaborn.despine()
plt.show()

In [ ]:
binarizedimage = croppedimage < threshold_iso  # porespy expects 'True' for features of interest, so we true stuff smaller than the threshold, e.g the air

In [ ]:
plt.subplot(121)
plt.imshow(croppedimage)
plt.title('Center of original slice, %s x %s px' % (croppedimage.shape[0], croppedimage.shape[1]))
plt.gca().add_artist(ScaleBar(voxelsize,'um'))
plt.axis('off')
plt.subplot(122)
plt.imshow(~binarizedimage) # Invert for displaying
plt.title('Binarized image, %s x %s px' % (binarizedimage.shape[0], binarizedimage.shape[1]))
plt.gca().add_artist(ScaleBar(voxelsize,'um'))
plt.axis('off')
plt.show()

In [ ]:
localthickness=ps.filters.local_thickness(binarizedimage, sizes=50)

In [ ]:
import numpy as np

In [ ]:
len(np.unique(localthickness))

In [ ]:
plt.subplot(131)
plt.imshow(localthickness, cmap='magma')
plt.gca().add_artist(ScaleBar(voxelsize,'um'))
plt.axis('off')
plt.subplot(132)
plt.imshow(localthickness/binarizedimage, cmap='magma')
plt.gca().add_artist(ScaleBar(voxelsize,'um'))
plt.axis('off')
plt.subplot(133)
plt.imshow(~binarizedimage)
plt.gca().add_artist(ScaleBar(voxelsize,'um'))
plt.axis('off')
plt.show()

In [ ]:
import scipy

In [ ]:
dt = scipy.ndimage.distance_transform_edt(binarizedimage)
distance = 100
peaks = skimage.feature.peak_local_max(dt, min_distance=distance)
# skeleton = skimage.morphology.skeletonize(skimage.morphology.dilation(binarizedimage))

In [ ]:
# plt.imshow(skeleton)

In [ ]:
plt.imshow(dt/binarizedimage, cmap='viridis')
plt.gca().add_artist(ScaleBar(voxelsize,'um'))
plt.axis('off')
for peak in peaks:
    plt.scatter(peak[1], peak[0], marker='x', c='white')
plt.title('Distance transformation with %s overlaid peaks > %s px' % (len(peaks), distance))
plt.show()

In [ ]:
pk = ps.filters.find_peaks(dt, r_max=50)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=[6, 3])

pk = ps.filters.find_peaks(dt=dt)
ax[0].imshow((dt/binarizedimage), cmap='viridis')
ax[0].axis(False)
plt.gca().add_artist(ScaleBar(voxelsize,'um'))

ax[1].imshow(dt/
             binarizedimage/
             ~skimage.morphology.dilation(skimage.morphology.dilation(skimage.morphology.dilation(pk))), cmap='viridis')
ax[1].axis(False);
plt.axis('off')
plt.show()

Pore size distribution as per https://nbviewer.org/github/PMEAL/porespy/blob/dev/examples/filters/tutorials/local_thickness.ipynb

In [ ]:
psd = ps.metrics.pore_size_distribution(localthickness,
                                        log=False,
                                        bins=25,
                                        voxel_size=1.5)  # give the voxel size in um, then we get back um :)
print(psd)

In [ ]:
plt.bar(x=psd.R, height=psd.pdf, width=psd.bin_widths)
plt.xlabel('Pore radius [um]')
plt.ylabel('Normalized volume fraction')

SNOW partitioning, based on https://nbviewer.org/github/PMEAL/porespy/blob/dev/examples/filters/tutorials/snow_partitioning.ipynb

In [ ]:
from skimage.morphology import binary_dilation
ps.visualization.set_mpl_style()
np.random.seed(1)

In [ ]:
snow_out = ps.filters.snow_partitioning(binarizedimage, r_max=40, sigma=0.4)
print(snow_out)

In [ ]:
dt_peak = snow_out.dt.copy()
peaks_dilated = binary_dilation(snow_out.peaks >> 0)
dt_peak[peaks_dilated > 0] = np.nan

In [ ]:
plt.subplot(221)
plt.imshow(~snow_out.im, cmap='gray')
plt.title('(inverted) Binary image')
plt.gca().add_artist(ScaleBar(voxelsize,'um'))
plt.axis('off')
plt.subplot(222)
plt.imshow(snow_out.dt)
plt.title('Distance transform')
plt.gca().add_artist(ScaleBar(voxelsize,'um'))
plt.axis('off')
plt.subplot(223)
plt.imshow(dt_peak)
plt.title("Distance transform peaks");
plt.gca().add_artist(ScaleBar(voxelsize,'um'))
plt.axis('off')
plt.subplot(224)
plt.imshow(ps.tools.randomize_colors(snow_out.regions)/binarizedimage)
plt.title("Segmentation")
plt.gca().add_artist(ScaleBar(voxelsize,'um'))
plt.axis('off')
plt.show()